In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('/content/final_aspect_sentiment_data.csv')

In [3]:
data.head()

,Unnamed: 0,asin,name,rating,date,verified,title_x,body,helpfulVotes,brand,product_title,image,totalReviews,price,originalPrice,row_id,aspect,sentiment_term,sentiment_category
0,0,B00F2SKPIM,Bikerhiker,5,"February 1, 2014",True,a fantastic product!!!,As good or better than I had hoped for. This i...,NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.0,0.0,1,design,fantastic,positive
1,1,B00F2SKPIM,Bikerhiker,5,"February 1, 2014",True,a fantastic product!!!,As good or better than I had hoped for. This i...,NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.0,0.0,1,cpu/gpu,crazy fast,positive
2,2,B00F2SKPIM,Bikerhiker,5,"February 1, 2014",True,a fantastic product!!!,As good or better than I had hoped for. This i...,NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.0,0.0,1,display,size,positive
3,3,B00F2SKPIM,JCinPC,5,"April 1, 2014",True,terrific phone,"Everyone at work laughs at me when I ""answer m...",NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.0,0.0,4,design,terrific,positive
4,4,B00F2SKPIM,JCinPC,5,"April 1, 2014",True,terrific phone,"Everyone at work laughs at me when I ""answer m...",NaN,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wir...",https://m.media-amazon.com/images/I/91eFtaIWpc...,983,0.0,0.0,4,battery,Charge the battery,neutral


In [20]:
data['price'].mean()

232.8721052631579

In [4]:
data.groupby(['brand']).agg({'product_title':'nunique'})

,product_title
brand,
Motorola,2
Nokia,1
Samsung,7


In [5]:
# Install LangChain package
!pip install langchain
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [8]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.9 MB/s eta 0:00:00


In [10]:
# Importing necessary libraries
import os
import json
import openai
import pandas as pd
from langchain.llms import OpenAI


In [31]:
# Set up OpenAI API key
#openai.api_key = open("api_key.txt", "r").read().strip()
os.environ["OPENAI_API_KEY"]=""


# Design recomendation

In [21]:
def start_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"
    example_user_req = {'camera': 'good','display': 'good','battery': 'very good','memory': 'good','cpu/gpu': 'very good','design':'very good','Price': '232.8'}

    system_message = f"""

    You are an intelligent cell phone expert and your goal is to find the best cell phone for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    Don't make assumptions about what values to plug into functions.

    Arrive at the function call in not more than five questions and answers

    {delimiter} Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a cell phone  that can handle demanding tasks. Hence, the cell phone should have High-quality camera for photos and videos.Long-lasting battery for extensive use.You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I edit the videos."
    Assistant: "Thank you for providing that information. So your work involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information.Could you kindly let me know your budget for the cell phone?This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 100 dollars"
    Assistant: <function call with correct values >
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation

In [25]:
user_intent_function = [
{
    "name": "user_intent_function",
    "description": "Get structured output from users cellphone requirement",
    "parameters": {
        "type": "object",
        "properties": {
            "Camera": {
                "type": "string",
                "description": "If user need good camera for photography",
                "enum" : ["very good", "good", "neutral"]
            },
            "CPU/GPU": {
                "type": "string",
                "description": "If user need GPU for gaming or editorial work",
                "enum" : ["very good", "good", "neutral"]
            },
            "Display": {
                "type": "string",
                "enum" : ["very good", "good", "neutral"]

            },
            "Battery": {
                "type": "string",
                "enum" : ["very good", "good", "neutral"]},

            "Memory": {
                "type": "string",
                "description": "Storage requirement of the cell phone",
                "enum" : ["high", "medium", "low"]
            },
            "Design": {
                "type": "string",
                "description": "User's need for sleek or latest design phone",
                "enum" : ["very good", "good", "neutral"]
            },

            "Price": {
                "type": "string",
                "description": "User's budget for buying the cell phone"
            },
        },
        "required": ["Camera", "CPU/GPU", "Display","Battery","Memory", "Design","Price"],
    },

}
]

In [29]:
def query_completions(messages):
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o",
        max_tokens = 800,
        messages=messages,
        functions=user_intent_function,
        function_call="auto",
        temperature = 0.5
    )
    return response.choices[0].message

In [39]:
conversation = start_conversation()
introduction = query_completions(conversation).content
print("Assistant: " + introduction + '\n')
user_input = ''
while(user_input != "exit"):
    user_input = input("")
    conversation.append({"role": "user", "content": user_input})
    response_assistant = query_completions(conversation)
    print("Assistant: " + response_assistant.content)



Assistant: Hello! I'm here to help you find the best cell phone that suits your needs. Could you please share what you primarily use your cell phone for? This will help me understand your requirements better.

I am a photographer
Assistant: Great! As a photographer, you likely need a cell phone with a high-quality camera, good storage capacity, and a reliable battery. 

To better understand your requirements, could you please share more details about your specific needs? For example, do you prioritize camera quality over other features, and do you often work with high-resolution photos that require substantial storage?
yes i need good camera quality and storage
Assistant: Great! As a photographer, having a cell phone with an excellent camera and ample storage is essential. 

To make the best recommendation, could you please provide more details on the following:
1. What is your budget for the cell phone?
2. Do you have any preferences for battery life, display quality, or design?

This

TypeError: can only concatenate str (not "NoneType") to str